In [5]:
import heapq
import math
from collections import Counter

# Функция для построения дерева Хаффмана
def build_huffman_tree(symbols):
    # Создаем приоритетную очередь
    heap = [[weight, [symbol, ""]] for symbol, weight in symbols]
    heapq.heapify(heap)

    # Строим дерево
    while len(heap) > 1:
        lo = heapq.heappop(heap)
        hi = heapq.heappop(heap)
        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]
        heapq.heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
    
    return sorted(heap[0][1:], key=lambda p: (len(p[-1]), p))

# Функция для кодирования по Хаффману
def huffman_encode(data):
    # Подсчет частоты символов
    frequency = Counter(data)
    
    # Сортировка символов по частоте
    sorted_symbols = sorted(frequency.items(), key=lambda x: x[1], reverse=True)
    
    # Строим дерево Хаффмана
    huffman_tree = build_huffman_tree(sorted_symbols)
    
    # Формируем код
    huffman_codes = {symbol: code for symbol, code in huffman_tree}
    
    # Кодирование данных
    encoded_data = ''.join([huffman_codes[symbol] for symbol in data])

    return huffman_codes, encoded_data, frequency

# Функция для построения дерева Шеннона-Фано
def build_shannon_fano_tree(symbols, codes, prefix=""):
    if len(symbols) == 1:
        # Базовый случай: если символ только один, назначаем ему код
        symbol = symbols[0][0]
        codes[symbol] = prefix
        return

    # Разделяем символы на две группы с примерно одинаковыми вероятностями
    total_weight = sum([item[1] for item in symbols])
    running_sum = 0
    split_point = 0
    for i, (symbol, weight) in enumerate(symbols):
        running_sum += weight
        if running_sum >= total_weight / 2:
            split_point = i + 1
            break

    # Рекурсивно строим коды для левой и правой групп
    build_shannon_fano_tree(symbols[:split_point], codes, prefix + "0")
    build_shannon_fano_tree(symbols[split_point:], codes, prefix + "1")

# Функция для кодирования по Шеннону-Фано
def shannon_fano_encode(data):
    # Подсчет частоты появления символов
    frequency = Counter(data)

    # Сортировка символов по частоте
    sorted_symbols = sorted(frequency.items(), key=lambda x: x[1], reverse=True)

    # Построение кодов Шеннона-Фано
    codes = {}
    build_shannon_fano_tree(sorted_symbols, codes)

    # Кодирование сообщения
    encoded_data = ''.join([codes[symbol] for symbol in data])

    return codes, encoded_data, frequency

# Функция для вычисления энтропии
def calculate_entropy(frequency):
    total = sum(frequency.values())
    entropy = 0
    for count in frequency.values():
        probability = count / total
        entropy -= probability * math.log2(probability)
    return entropy

# Функция для вычисления средней длины кодирования
def calculate_average_code_length(codes, frequency):
    total_symbols = sum(frequency.values())
    avg_length = 0
    for symbol, code in codes.items():
        avg_length += len(code) * frequency[symbol]
    return avg_length / total_symbols

# Пример использования
if __name__ == "__main__":
    # Входные данные (25 числовых значений с повторениями)
    data = [5, 9, 12, 15, 9, 25, 9, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 120]
    
    # Хаффман
    huffman_codes, huffman_encoded_data, huffman_frequency = huffman_encode(data)
    print("Хаффман: Закодированный алфавит (в битах):", huffman_codes)
    print("Хаффман: Закодированные данные:", huffman_encoded_data)
    huffman_avg_length = calculate_average_code_length(huffman_codes, huffman_frequency)
    print(f"Хаффман: Средняя длина кодирования (Lcp): {huffman_avg_length:.3f} бит")
    
    # Шеннон-Фано
    shannon_fano_codes, shannon_fano_encoded_data, shannon_fano_frequency = shannon_fano_encode(data)
    print("Шеннон-Фано: Закодированный алфавит (в битах):", shannon_fano_codes)
    print("Шеннон-Фано: Закодированные данные:", shannon_fano_encoded_data)
    shannon_fano_avg_length = calculate_average_code_length(shannon_fano_codes, shannon_fano_frequency)
    print(f"Шеннон-Фано: Средняя длина кодирования (Lcp): {shannon_fano_avg_length:.3f} бит")
    
    # Сравнение эффективности
    print(f"Эффективность (разница Lcp): {abs(huffman_avg_length - shannon_fano_avg_length):.3f} бит")
    
    # Предел энтропии
    huffman_entropy = calculate_entropy(huffman_frequency)
    shannon_fano_entropy = calculate_entropy(shannon_fano_frequency)
    print(f"Хаффман: Предел энтропии для источника: {huffman_entropy:.3f} бит")
    print(f"Шеннон-Фано: Предел энтропии для источника: {shannon_fano_entropy:.3f} бит")


Хаффман: Закодированный алфавит (в битах): {9: '011', 90: '0000', 95: '0001', 100: '0010', 105: '0011', 110: '0100', 120: '0101', 5: '10000', 12: '10001', 15: '10010', 25: '10011', 30: '10100', 35: '10101', 40: '10110', 45: '10111', 50: '11000', 55: '11001', 60: '11010', 65: '11011', 70: '11100', 75: '11101', 80: '11110', 85: '11111'}
Хаффман: Закодированные данные: 10000011100011001001110011011101001010110110101111100011001110101101111100111011111011111000000010010001101000101
Хаффман: Средняя длина кодирования (Lcp): 4.520 бит
Шеннон-Фано: Закодированный алфавит (в битах): {9: '0000', 5: '0001', 12: '00100', 15: '00101', 25: '0011', 30: '01000', 35: '01001', 40: '0101', 45: '01100', 50: '01101', 55: '0111', 60: '10000', 65: '10001', 70: '1001', 75: '10100', 80: '10101', 85: '1011', 90: '11000', 95: '11001', 100: '1101', 105: '11100', 110: '11101', 120: '1111'}
Шеннон-Фано: Закодированные данные: 00010000001000010100000011000001000010010101011000110101111000010001100110100101011011110